In [1]:
import numpy as np
import pandas as pd
import scipy.constants as consts
import matplotlib.pyplot as plt
from io import StringIO
import glob
import os, sys
import re
plt.rcParams['figure.dpi']= 300

In [2]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 5),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)
# set wanted parameters for plots

# e) Interaction case

Read in energies:

In [27]:
def name_dat_file(directory):
    filenames = [f for f in glob.glob(directory + '*.dat') if 'Eint' in f]
    return filenames

In [28]:
def name_dat_filemain(directory):
    filenames = [f for f ]

SyntaxError: unexpected EOF while parsing (<ipython-input-28-84a5f9a6893a>, line 1)

Sort the filenames nicely:

In [4]:
def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]"""
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    #Sort the given list in the way that humans expect
    l.sort(key=alphanum_key)
    return l

## Read main short file in to a dataframe:

In [5]:
def matchlist_iter(filename):
    column_names = ['energy', 'acceptance', 'timecpu','solver']
    df = pd.read_csv(filename,skiprows=5,names=column_names,delim_whitespace=True,comment='#')
    return df

## Read in energy files:

In [6]:
def read_Ean_2_df(filenamesmain,filenamean,filenamenum,Ns,dims=1):
    newDF = pd.DataFrame() #creates a new dataframe that's empty
    mainDF = pd.DataFrame()
    column_names = [['energy_num'],['energy_an']]
    print(Ns)
    energy = pd.DataFrame()
    for i in range(len(filenamesmain)):
        temp = pd.DataFrame()
        main = matchlist_iter(filenamesmain[i])
        main = main.assign(N=Ns[i]*np.ones(len(main)))
        main = main.assign(dim=dims * np.ones(len(main)))
        temp['E_an'] = pd.read_csv(filenamean[i])
        temp = temp.assign(N=Ns[i]*np.ones(len(temp)))
        temp = temp.assign(dim=dims * np.ones(len(temp)))
        mainDF = mainDF.append(main,ignore_index=True)
        temp['E_n'] = pd.read_csv(filenamenum[i],delim_whitespace=True,usecols=[0])
        energy = energy.append(temp,ignore_index=True)
    return mainDF, energy


### Make the dataframe drop uneccesary columns

In [7]:
def col_on_u(data):
    newcols = ['alpha', 'E_mean_an', 'std_b','acceptance', 'timecpu']
    return data[newcols]
 

## Blocking

In [8]:
def block_mean(vec):
    return sum(vec)/len(vec)

def meanAndVariance(vec):
    mean = np.mean(vec)
    var = sum([i ** 2 for i in vec])/len(vec) - mean*mean
    return mean, var

In [9]:
def everything_is_awesome(data): # does the blocking on the dataframe
    n_blocks = 200
    block_size_min = 100
    block_size_max = len(data)/100
    block_step = int ((block_size_max - block_size_min + 1) / n_blocks)
    mean_vec = []
    var_vec = []
    block_sizes = []
    for i in range(0, n_blocks):
        mean_temp_vec = []
        start_point = 0
        end_point = block_size_min + block_step*i
        block_size = end_point
        block_sizes.append(block_size)

    mean_temp_vec.append(block_mean(data[start_point:end_point]))
    start_point = end_point
    end_point += block_size_min + block_step*i
    mean, var = meanAndVariance(mean_temp_vec)
    mean_vec.append(mean)
    var_vec.append(np.sqrt(var/(len(data)/float(block_size) - 1.0)))

    mean, var = meanAndVariance(data)

    return mean,var

## Make $\LaTeX$ tables

In [10]:
def latex(df,colname):
    df = df.rename(columns=dict(zip(df, colname)))
    table = df.to_latex(index=False,escape=False,column_format=(1+ len(colname))*'c')
    table = table.replace("toprule", "hline \hline")
    table = table.replace("bottomrule", "hline \hline")
    table = table.replace("midrule", "hline")
    s = r'''\begin{table}[H]
    \centering
    \caption{}
    \label{tab:}
    '''
    table = s + table 
    table = table +'\end{table}'
    return table

filepath for e)

In [20]:
filepath = ['e/e_a0/','e/e_n10_alphas/','e/e_n50_alphas/','e/e_n100_alphas/']

find all .dat files

In [21]:
files = []
for i in range(len(filepath)):
    files.append(name_dat_file(filepath[i]))


[['e/e_a0/Eint_a0_e_a0_n10_N10.dat',
  'e/e_a0/Eint_a0_e_a0_n50_N50.dat',
  'e/e_a0/e_a0_n100_alpha0.500000.dat',
  'e/e_a0/e_a0_n10.dat',
  'e/e_a0/e_a0_n50.dat',
  'e/e_a0/Eint_e_a0_n100_alpha0.500000.dat'],
 [],
 ['e/e_n50_alphas/e_n50_alpha0.600000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.400000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.300000.dat',
  'e/e_n50_alphas/e_n50_alpha0.400000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.600000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.700000.dat',
  'e/e_n50_alphas/e_n50_alpha0.700000.dat',
  'e/e_n50_alphas/e_n50_alpha0.300000.dat'],
 ['e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.700000.dat',
  'e/e_n100_alphas/e_n100_alpha0.600000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.300000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.400000.dat',
  'e/e_n100_alphas/e_n100_alpha0.300000.dat',
  'e/e_n100_alphas/e_n100_alpha0.400000.dat',
  'e/e_n100_alphas/e_n100_alpha0.700000.dat',
  'e/e_n100_alphas/Eint_a_e_n1

In [25]:
listfiles = []
for k in range(len(files)):
    sort_nicely(files[k])

[['e/e_a0/Eint_a0_e_a0_n10_N10.dat',
  'e/e_a0/Eint_a0_e_a0_n50_N50.dat',
  'e/e_a0/Eint_e_a0_n100_alpha0.500000.dat',
  'e/e_a0/e_a0_n10.dat',
  'e/e_a0/e_a0_n50.dat',
  'e/e_a0/e_a0_n100_alpha0.500000.dat'],
 [],
 ['e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.300000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.400000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.600000.dat',
  'e/e_n50_alphas/Eint_a_e_n50_N50_alpha0.700000.dat',
  'e/e_n50_alphas/e_n50_alpha0.300000.dat',
  'e/e_n50_alphas/e_n50_alpha0.400000.dat',
  'e/e_n50_alphas/e_n50_alpha0.600000.dat',
  'e/e_n50_alphas/e_n50_alpha0.700000.dat'],
 ['e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.300000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.400000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.600000.dat',
  'e/e_n100_alphas/Eint_a_e_n100_N100_alpha0.700000.dat',
  'e/e_n100_alphas/e_n100_alpha0.300000.dat',
  'e/e_n100_alphas/e_n100_alpha0.400000.dat',
  'e/e_n100_alphas/e_n100_alpha0.600000.dat',
  'e/e_n100_alphas